# Solving Sudoku Puzzles
## Assignment Preamble
Please ensure you carefully read all of the details and instructions on the assignment page, this section, and the rest of the notebook. If anything is unclear at any time please post on the forum or ask a tutor well in advance of the assignment deadline.

In addition to all of the instructions in the body of the assignment below, you must also follow the following technical instructions for all assignments in this unit. *Failure to do so may result in a grade of zero.*
* [At the bottom of the page](#Submission-Test) is some code which checks you meet the submission requirements. You **must** ensure that this runs correctly before submission.
* Do not modify or delete any of the cells that are marked as test cells, even if they appear to be empty.
* Do not duplicate any cells in the notebook – this can break the marking script. Instead, insert a new cell (e.g. from the menu) and copy across any contents as necessary.

Remember to save and backup your work regularly, and double-check you are submitting the correct version.

This notebook is the primary reference for your submission. You may write code in separate `.py` files but it must be clearly imported into the notebook so that it runs without needing to reference those files, and you must explain clearly what functionality is contained in those files (through comments, markdown cells, etc).

As always, **the work you submit for this assignment must be entirely your own.** Do not copy or work with other students. Do not copy answers that you find online. These assignments are designed to help improve your understanding first and foremost – the process of doing the assignment is part of *learning*. They are also used to assess your ability, and so you must uphold academic integrity. Submitting plagiarised work risks your entire place on your degree.

**The pass mark for this assignment is 40%.** We expect that students, on average, will be able to produce a submission which gets a mark between 50-70% within the normal workload allocation for the unit, but this will vary depending on individual backgrounds. Please ask for help if you are struggling.

## Getting Started
For this assignment, you will be writing an agent that can solve sudoku puzzles. You should be familiar with sudoku puzzles from the unit material. You are given a 9x9 grid with some fixed values. To solve the puzzle, the objective is to fill the empty cells of the grid such that the numbers 1 to 9 appear exactly once in each row, column, and 3x3 block of the grid. 

Below is a sample puzzle along with its solution. 

<img src="images/sudoku.png" style="width: 50%;"/>

For this assignment you will need to submit:
1. The implementation for an agent which can solve sudoku puzzles – this notebook
 * You can use any algorithm you like, from the unit material or otherwise
 * Your code will be subject to automated testing, from which grades will be assigned based on whether it can solve sudokus of varying difficulty
 * To get a high grade on this assignment, the speed of your code will also be a factor – the quicker the better
 * There are some sample tests included below, make sure your code is compatible with the format of these tests
2. A text file that explains your approach and the decisions you made in your own words – a readme file
 * Submissions that do not include the written section will receive zero marks – **this part is mandatory**
 * You may write your file in plain text (.txt) or [Markdown](https://www.markdownguide.org/basic-syntax/) (.md)
 * To get top marks on this assignment, as well as getting a high grade from your implementation, you must also demonstrate excellent academic presentation in your written section

### Choice of Algorithm
The choice of algorithm to solve sudoku puzzles is up to you. We expect you will use search techniques from the unit, but you could make something up yourself, or do some independent research to find something else. You will need to evaluate and balance the trade-off between how well suited you think the algorithm is and how difficult it is to write, but there is some advice below.

I suggest you implement *constraint satisfaction* as it is described in the unit material. You can use the code you have previously been given as a guide. A good implementation of a backtracking depth-first search with constraint propagation should be sufficient to get a good grade in the automated tests (roughly 60-70%).

You could also write a successful agent that uses the other search techniques you have seen in the unit so far: basic search, heuristic search, or local search. You may find these easier to implement, though they may perform less well. 

To get a high grade on this assignment will require a particularly efficient implementation of constraint satisfaction, or something which goes beyond the material we have presented. *This is left unguided and is not factored into the unit workload estimates.*

If you choose to implement more than one algorithm, please feel free to include your code and write about it in part two (readme file), but only the code in this notebook will be used in the automated testing.

## Sample Sudoku Puzzles
To get started, the cell below will load in some sample sudoku puzzles for you so you can see the format. There are sudokus provided of multiple difficulties (easier sudokus typically start with more digits provided). The cell below only loads the easiest, but there is another test cell lower in the notebook which will run your code against all of the provided puzzles.

Each sudoku is a 9x9 NumPy array of integers, where zero represents an empty square. Each difficulty comes with 15 sudokus, so when you load the file, it is stored in a 15x9x9 array.

In [1]:
import numpy as np

# Load sudokus
sudoku = np.load("data/very_easy_puzzle.npy")
print("very_easy_puzzle.npy has been loaded into the variable sudoku")
print(f"sudoku.shape: {sudoku.shape}, sudoku[0].shape: {sudoku[0].shape}, sudoku.dtype: {sudoku.dtype}")

# Load solutions for demonstration
solutions = np.load("data/very_easy_solution.npy")
print()

# Print the first 9x9 sudoku...
print("First sudoku:")
print(sudoku[0], "\n")

# ...and its solution
print("Solution of first sudoku:")
print(solutions[0])

very_easy_puzzle.npy has been loaded into the variable sudoku
sudoku.shape: (15, 9, 9), sudoku[0].shape: (9, 9), sudoku.dtype: int8

First sudoku:
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]] 

Solution of first sudoku:
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]


## Part One
You should write all of your code for solving sudokus below this cell.

You must include a function called `sudoku_solver(sudoku)` which takes one sudoku puzzle (a 9x9 NumPy array) as input, and returns the solved sudoku as another 9x9 NumPy array. This is the function which will be tested. 

In [2]:
# A cell class for constraint propagation
class Cell:
    def __init__(self, current, r, c, found, value, possible_numbers):
        self.current = current
        self.r = r
        self.c = c
        self.found = found
        self.value = value
        self.possible_numbers = possible_numbers


# Checks the initial state of the sudoku
def check_initial_state(sudoku):
    for r in range(0, 9):
        for c in range(0, 9):
            number = sudoku[r][c]
            if number != 0:
                sudoku[r][c] = -1
                if not is_ok(r, c, number, sudoku):
                    return False
                sudoku[r][c] = number
    return True


# Checks if the number is in that row of the sudoku
def is_in_row(row, number, sudoku):
    for i in range(0, 9):
        if sudoku[row][i] == number:
            return True
    return False


# Checks if the number is in that col of the sudoku
def is_in_col(col, number, sudoku):
    for i in range(0, 9):
        if sudoku[i][col] == number:
            return True
    return False


# Checks if the number is in the smaller 3x3 grid of that row and col of the sudoku
def is_in_box(row, col, number, sudoku):
    r = row - row % 3
    c = col - col % 3

    for i in range(r, r + 3):
        for j in range(c, c + 3):
            if sudoku[i][j] == number:
                return True

    return False


# Returns true if the number can be placed at that row and col of the sudoku
def is_ok(row, col, number, sudoku):
    return (
        not (is_in_row(row, number, sudoku)) and not (is_in_col(col, number, sudoku)) and 
        not (is_in_box(row, col, number, sudoku))
    )

def sudoku_solver(sudoku):
    """
    Solves a Sudoku puzzle and returns its unique solution.

    Input
        sudoku : 9x9 numpy array
            Empty cells are designated by 0.

    Output
        9x9 numpy array of integers
            It contains the solution, if there is one. If there is no solution, all array entries should be -1.
    """

    ### YOUR CODE HERE

    if not check_initial_state(sudoku):
        return np.full((9, 9), -1)

    if solve(sudoku):
        return sudoku
    else:
        return np.full((9, 9), -1)


# SEARCH ALGORITHM

def solve(sudoku):
    # Constrain the search space
    reduce_sudoku(sudoku)

    # In case during constraint propagation an invalid value has been put in
    if not check_initial_state(sudoku):
        return False

    # Loop through every cell in the sudoku
    for r in range(0, 9):
        for c in range(0, 9):
            # If it's 0
            if sudoku[r][c] == 0:
                for number in range(1, 10):
                    # Check if any numbers can be put in the sudoku
                    if is_ok(r, c, number, sudoku):
                        # Create copy of current state of sudoku for backtracking
                        old_sudoku = copy_sudoku(sudoku)

                        # If the number works put it in the soduku as a guess
                        sudoku[r][c] = number

                        # Solve the current state of the sudoku
                        if solve(sudoku):
                            return True
                        # Backtrack if it doesn't work and reset values
                        else:
                            backtrack(sudoku, old_sudoku)
                            # Free up memory space
                            del old_sudoku

                # If no possible numbers return false
                return False
    # If reaches the end sudoku is solved
    return True

# MAKE COPY OF SUDOKU

# This is slightly faster than copy.deepcopy
def copy_sudoku(sudoku):
    copied_sudoku = np.empty([9, 9], dtype=int)
    for r in range(0, 9):
        for c in range(0, 9):
            copied_sudoku[r][c] = sudoku[r][c]
    return copied_sudoku

# BACKTRACK

# So that the search algorithm doesn't return the wrong object
def backtrack(sudoku, old_sudoku):
    for r in range(0, 9):
        for c in range(0, 9):
            sudoku[r][c] = old_sudoku[r][c]

# TURN NUMBER BASED SUDOKU TO CELL BASED SUDOKU

def make_cell_sudoku(sudoku):
    cell_sudoku = np.empty([9, 9], dtype=Cell)
    solved_values = 0

    for r in range(0, 9):
        for c in range(0, 9):
            # Value needs to be found
            if sudoku[r][c] == 0:
                numbers = []
                for i in range(1, 10):
                    numbers.append(i)
                temp = Cell(False, r, c, False, 0, numbers)
                cell_sudoku[r][c] = temp
            # Value is already found
            else:
                numbers = []
                temp = Cell(False, r, c, True, sudoku[r][c], numbers)
                cell_sudoku[r][c] = temp
                solved_values += 1

    return cell_sudoku, solved_values


# CONSTRAINT PROPAGATION

def reduce_sudoku(sudoku):
    cell_sudoku, solved_values = make_cell_sudoku(sudoku)

    stalled = False
    while not stalled:
        solved_values_before = solved_values

        # Row
        eliminate(cell_sudoku)
        stalled, solved_values = single_possible_value_finder(cell_sudoku, sudoku, solved_values, False)
        eliminate(cell_sudoku)
        only_choice_row(cell_sudoku)

        # Col
        eliminate(cell_sudoku)
        stalled, solved_values = single_possible_value_finder(cell_sudoku, sudoku, solved_values, False)
        eliminate(cell_sudoku)
        only_choice_col(cell_sudoku)

        # Box
        eliminate(cell_sudoku)
        stalled, solved_values = single_possible_value_finder(cell_sudoku, sudoku, solved_values, False)
        eliminate(cell_sudoku)
        only_choice_box(cell_sudoku)

        eliminate(cell_sudoku)
        stalled, solved_values = single_possible_value_finder(cell_sudoku, sudoku, solved_values, True)
        eliminate(cell_sudoku)

        # If we can't do anything then we are stalled
        if not stalled:
            stalled = solved_values == solved_values_before


# FIND CELLS WITH ONLY ONE POSSIBLE VALUE

def single_possible_value_finder(cell_sudoku, sudoku, solved_values, check_state):
    for r in range(0, 9):
        for c in range(0, 9):
            if len(cell_sudoku[r][c].possible_numbers) == 1:
                cell_sudoku[r][c].found = True
                cell_sudoku[r][c].value = cell_sudoku[r][c].possible_numbers[0]
                cell_sudoku[r][c].possible_numbers.clear()

                sudoku[r][c] = cell_sudoku[r][c].value

                solved_values += 1

                # Will be true for last call of this method
                if check_state:
                    # If our current state is not okay
                    if not check_initial_state(sudoku):
                        # Set stalled to true
                        return True, solved_values

                return False, solved_values

    # Means sudoku is solved
    return True, solved_values


# ELIMINATE NOT POSSIBLE NUMBERS FROM CELLS

def eliminate(cell_sudoku):
    for r in range(0, 9):
        for c in range(0, 9):
            if cell_sudoku[r][c].found == False:
                already_have_numbers = []

                # Remove non possible numbers from the row and column
                for i in range(0, 9):
                    if cell_sudoku[r][i].found == True:
                        already_have_numbers.append(cell_sudoku[r][i].value)
                    if cell_sudoku[i][c].found == True:
                        already_have_numbers.append(cell_sudoku[i][c].value)

                # Remove non possible numbers from the 3x3 grid
                r0 = r - r % 3
                c0 = c - c % 3
                for i in range(r0, r0 + 3):
                    for j in range(c0, c0 + 3):
                        if cell_sudoku[i][j].found == True:
                            already_have_numbers.append(cell_sudoku[i][j].value)

                for i in range(0, len(already_have_numbers)):
                    if already_have_numbers[i] in cell_sudoku[r][c].possible_numbers:
                        cell_sudoku[r][c].possible_numbers.remove(already_have_numbers[i])

                already_have_numbers.clear()


# CHECK THE ONLY SUITABLE NUMBERS FOR A CELL

# The only choice for the row
def only_choice_row(cell_sudoku):
    for r in range(0, 9):
        for c in range(0, 9):
            should_continue = True
            # If current cell not solved
            if cell_sudoku[r][c].found == False:
                cell_sudoku[r][c].current = True
                # Check it's possible numbers
                for i in range(0, len(cell_sudoku[r][c].possible_numbers)):
                    if should_continue:
                        # Will look at every non-found cell's possible numbers in this row
                        for row in range(0, 9):
                            if (cell_sudoku[r][row].found == False) and (cell_sudoku[r][row].current == False):
                                # If they could both be i break 
                                if cell_sudoku[r][c].possible_numbers[i] in cell_sudoku[r][row].possible_numbers:
                                    break

                            # If no other cells in the row can be i, then this cell has to be i
                            if row == 8:
                                cell_sudoku[r][c].current = False
                                cell_sudoku[r][c].possible_numbers = [cell_sudoku[r][c].possible_numbers[i]]
                                should_continue = False
                    else:
                        break
                cell_sudoku[r][c].current = False

# The only choice for the column
def only_choice_col(cell_sudoku):
    for r in range(0, 9):
        for c in range(0, 9):
            should_continue = True
            # If current cell not solved
            if cell_sudoku[r][c].found == False:
                cell_sudoku[r][c].current = True
                # Check it's possible numbers
                for i in range(0, len(cell_sudoku[r][c].possible_numbers)):
                    if should_continue:
                        # Will look at every non-found cell's possible numbers in this column
                        for col in range(0, 9):
                            if (cell_sudoku[col][c].found == False) and (cell_sudoku[col][c].current == False):
                                # If they could both be i break 
                                if cell_sudoku[r][c].possible_numbers[i] in cell_sudoku[col][c].possible_numbers:
                                    break

                            # If no other cells in the column can be i, then this cell has to be i
                            if col == 8:
                                cell_sudoku[r][c].current = False
                                cell_sudoku[r][c].possible_numbers = [cell_sudoku[r][c].possible_numbers[i]]
                                should_continue = False
                    else:
                        break
                cell_sudoku[r][c].current = False

# The only choice for the box
def only_choice_box(cell_sudoku):
    for r in range(0, 9):
        for c in range(0, 9):
            should_continue2 = True
            # If current cell not solved
            if cell_sudoku[r][c].found == False:
                cell_sudoku[r][c].current = True
                # Check it's possible numbers
                for i in range(0, len(cell_sudoku[r][c].possible_numbers)):
                    if should_continue2:
                        r0 = r - r % 3
                        c0 = c - c % 3
                        should_continue = True
                        # Will look at every non-found cell's possible numbers in this box
                        for row in range(r0, r0 + 3):
                            for col in range(c0, c0 + 3):
                                if should_continue:
                                    # If they could both be i break 
                                    if (
                                        (cell_sudoku[row][col].found == False) 
                                        and (cell_sudoku[row][col].current == False)
                                    ):
                                        if (
                                            cell_sudoku[r][c].possible_numbers[i] in 
                                            cell_sudoku[row][col].possible_numbers
                                        ):
                                            should_continue = False
                                            break
                                    # If no other cells in the box can be i, then this cell has to be i
                                    if (row == r + 2) and (col == c + 2):
                                        cell_sudoku[r][c].current = False
                                        cell_sudoku[r][c].possible_numbers = [cell_sudoku[r][c].possible_numbers[i]]
                                        should_continue2 = False

                    else:
                        break
                cell_sudoku[r][c].current = False

All of your code must go above this cell. You may add additional cells into the notebook if you wish, but do not duplicate or copy/paste cells as this can interfere with the grading script.

### Testing Details
There are four difficulties of sudoku provided: very easy, easy, medium, and hard. There are 15 sample sudokus in each category, with solutions as well. Difficulty was determined using reference solvers, but your code may vary; it is conceivable that your code will find some sudokus much easier or harder within a given category, or even between categories.

*All categories that are easy and above will contain* ***invalid initial states***, that is, sudoku puzzles with no solution. In this case, your function should return a 9x9 NumPy array whose values are all equal to -1.

When we test your code, we will firstly test it on the *same* very easy puzzles that you have been given. Then we will test it on additional *hidden* sudokus from each difficulty in turn, easy and up. Grades are awarded based on whether your code can solve the puzzles. For high grades on the hard puzzles, execution time will also be a factor. 

All puzzles must take under 30 seconds each on the test machine to count as successful, but you should be aiming for an average of under a second per puzzle. Hardware varies, but all tests will take place on the same modern desktop machine. Our ‘standard constraint satisfaction’ implementation takes about 0.001 seconds per puzzle for the very easy category, but struggles to solve some of the hard puzzles within the time limit.

***The hard sudokus are labelled as hard for a reason.*** We expect most submissions will not be able to solve them in a reasonable length of time. Use the stop button (■) on the toolbar if you need to terminate your code because it is taking too long.

The best way to improve the performance of your code is through a detailed understanding and smart choice of AI algorithms. This assignment is ***not*** meant to test your ability to write multi-threaded code or any other kind of high-performance code optimisations. 

#### Test Cell
The following code will run your solution over the provided sudoku puzzles. To enable it, set the constant `SKIP_TESTS` to `False`. If you fail any tests of one difficulty, the code will stop, but you can modify this behaviour if you like.

**IMPORTANT**: you must set `SKIP_TESTS` back to `True` before submitting this file!

In [3]:
SKIP_TESTS = True

if not SKIP_TESTS:
    import time
    difficulties = ['very_easy', 'easy', 'medium', 'hard']

    for difficulty in difficulties:
        print(f"Testing {difficulty} sudokus")
        
        sudokus = np.load(f"data/{difficulty}_puzzle.npy")
        solutions = np.load(f"data/{difficulty}_solution.npy")
        
        count = 0
        for i in range(len(sudokus)):
            sudoku = sudokus[i].copy()
            print(f"This is {difficulty} sudoku number", i)
            print(sudoku)
            
            start_time = time.process_time()
            your_solution = sudoku_solver(sudoku)
            end_time = time.process_time()
            
            print(f"This is your solution for {difficulty} sudoku number", i)
            print(your_solution)
            
            print("Is your solution correct?")
            if np.array_equal(your_solution, solutions[i]):
                print("Yes! Correct solution.")
                count += 1
            else:
                print("No, the correct solution is:")
                print(solutions[i])
            
            print("This sudoku took", end_time-start_time, "seconds to solve.\n")

        print(f"{count}/{len(sudokus)} {difficulty} sudokus correct")
        if count < len(sudokus):
            break

## Submission Test
The following cell tests if your notebook is ready for submission. **You must not skip this step!**

Restart the kernel and run the entire notebook (Kernel → Restart & Run All). Now look at the output of the cell below. 

*If there is no output, then your submission is not ready.* Either your code is still running (did you forget to skip tests?) or it caused an error.

As previously mentioned, failing to follow these instructions can result in a grade of zero.

In [4]:
import sys
import pathlib

fail = False;

if not SKIP_TESTS:
    fail = True;
    print("You must set the SKIP_TESTS constant to True in the cell above.")
    
p1 = pathlib.Path('./readme.txt')
p2 = pathlib.Path('./readme.md')
if not (p1.is_file() or p2.is_file()):
    fail = True;
    print("You must include a separate file called readme.txt or readme.md in your submission.")
    
p3 = pathlib.Path('./sudoku.ipynb')
if not p3.is_file():
    fail = True
    print("This notebook file must be named sudoku.ipynb")
    
if "sudoku_solver" not in dir():
    fail = True;
    print("You must include a function called sudoku_solver which accepts a numpy array.")
else: 
    sudoku = np.load("data/very_easy_puzzle.npy")[0]
    solution = np.load("data/very_easy_solution.npy")[0]

    if not np.array_equal(sudoku_solver(sudoku), solution):
        print("Warning:")
        print("Your sudoku_solver function does not correctly solve the first sudoku.")
        print()
        print("Your assignment is unlikely to get any marks from the autograder. While we will")
        print("try to check it manually to assign some partial credit, we encourage you to ask")
        print("for help on the forum or directly to a tutor.")
        print()
        print("Please use the readme file to explain your code anyway.")
    
if fail:
    print()
    sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")
else:
    print("All checks passed. When you are ready to submit, upload the notebook and readme file to the")
    print("assignment page, without changing any filenames.")
    print()
    print("If you need to submit multiple files, you can archive them in a .zip file. (No other format.)")

All checks passed. When you are ready to submit, upload the notebook and readme file to the
assignment page, without changing any filenames.

If you need to submit multiple files, you can archive them in a .zip file. (No other format.)


In [5]:
# This is a TEST CELL. Do not delete or change.